In [22]:
import pandas as pd
import json
import numpy as np

In [23]:
data = json.load(open("raw_comment_responses.json"))

In [24]:
comment_data = data['comments']

In [25]:
comment_data[0]

{'XmjsLyTW5EY': {'kind': 'youtube#commentThreadListResponse',
  'etag': 'f1pTg6SGlAcXCPmEF2dr_psBjlg',
  'nextPageToken': 'QURTSl9pMkdQQzVBR2J6SU5QdUhVOFgyYlJOQWdjd1E4MjlqWlNJa0lZUU5WdjNLN1poalRMRHFFYnpTSFFTbFRPZVN2UjNVbkFyRFN0RHk2SXRWUHNWTHBvZnN3N1Ra',
  'pageInfo': {'totalResults': 100, 'resultsPerPage': 100},
  'items': [{'kind': 'youtube#commentThread',
    'etag': 'SfpSPl1vQru2PxdGDqOaSXcZq8E',
    'id': 'UgxZzheIRuMB4FlmcBt4AaABAg',
    'snippet': {'videoId': 'XmjsLyTW5EY',
     'topLevelComment': {'kind': 'youtube#comment',
      'etag': 'RRwsukuerS12v6MG4pdXDWBkAWI',
      'id': 'UgxZzheIRuMB4FlmcBt4AaABAg',
      'snippet': {'videoId': 'XmjsLyTW5EY',
       'textDisplay': 'As many are pointing out in the comments, there&#39;s a lot of angles on this subject that were left out of this video. That&#39;s a fair point. I was trying to focus mostly on how the pandemic will end, not cover every piece of info about it. The vaccine process, all the various treatments, social implicati

In [26]:
videoIds = [list(comment.keys())[0] for comment in comment_data]

In [27]:
# Pass in a video (JSON response from YouTube API) return a dataframe
def getCommentsFromVideo(video):
    video_id = list(video.keys())[0]
    comments = list(video.values())[0]
    if 'noComments' in comments.keys():
        return
        # return pd.DataFrame({
        #     'videoId': video_id,
        #     'textDisplay': "",
        #     'textOriginal': "",
        #     'authorDisplayName': "",
        #     'authorProfileImageUrl': "",
        #     'authorChannelUrl':"",
        #     'authorChannelId':{'value':""},
        #     'canRate':False,
        #     'viewerRating':"",
        #     'likeCount':0,
        #     'publishedAt':"",
        #     'updatedAt':"",
        #     'commentId':""
        # })
    else:
        comments = comments['items']

    if len(comments) == 0:
        return
        # return pd.DataFrame({
        #     'videoId': video_id,
        #     'textDisplay': "",
        #     'textOriginal': "",
        #     'authorDisplayName': "",
        #     'authorProfileImageUrl': "",
        #     'authorChannelUrl':"",
        #     'authorChannelId':{'value':""},
        #     'canRate':False,
        #     'viewerRating':"",
        #     'likeCount':0,
        #     'publishedAt':"",
        #     'updatedAt':"",
        #     'commentId':""
        # })
    
    comment_data = []
    for comment in comments:
        datum = comment['snippet']['topLevelComment']
        comment_id = datum['id']
        comment_info = datum['snippet']
        comment_info['commentId'] = comment_id
        comment_data.append(comment_info)
    
    return pd.DataFrame(comment_data)

In [28]:
# Pass in a list of videos (JSONs) and return a concatenated dataframe. Pass in 1000 video (100000 comments)
def loadVideos(data):
    dataframes = []
    for video in data:
        dataframes.append(getCommentsFromVideo(video))
    
    return pd.concat(dataframes, ignore_index=True)

In [29]:
commentDf = pd.DataFrame()

In [30]:
currentlyDownloaded = len(comment_data)
currentlyTransformed = len(commentDf)
currentlyDownloaded, currentlyTransformed

(5926, 0)

In [31]:
first_1000 = loadVideos(comment_data[:1000])

In [32]:
second_1000 = loadVideos(comment_data[1000:2000])

In [33]:
third_1000 = loadVideos(comment_data[2000:3000])

In [34]:
fourth_1000 = loadVideos(comment_data[3000:4000])

In [35]:
fifth_1000 = loadVideos(comment_data[4000:5000])

In [36]:
sixth_1000 = loadVideos(comment_data[5000:6000])

In [37]:
dataHalf = pd.concat([first_1000,second_1000,third_1000,fourth_1000,fifth_1000,sixth_1000])

remainder = loadVideos(comment_data[6000:])

dataFull = pd.concat[[first_1000,second_1000,third_1000,fourth_1000,fifth_1000,sixth_1000,remainder]]

In [38]:
commentsGrouped = pd.DataFrame(dataHalf[['videoId', 'textOriginal']].groupby('videoId')['textOriginal'].apply(list)).reset_index()

In [39]:
# commentsGrouped['_id'] = mongoId
# commentsGrouped = commentsGrouped.rename(columns={"_id":"_id", "textOriginal":"comments", "videoId":"id"}).reindex(columns = ['_id', 'comments', 'id'])
commentsGrouped = commentsGrouped.rename(columns={"textOriginal":"comments", "videoId":"id"}).reindex(columns = ['comments', 'id'])

In [40]:
commentsGrouped.iloc[2]

comments    [Bully hunters!...gay. You all better stay awa...
id                                                -1L9MM_NEVg
Name: 2, dtype: object

In [41]:
commentsJson = commentsGrouped.to_json(orient = 'records')
with open("../pseudoscience-paper-data/groundtruth_videos_comments.json", "w") as f:
    f.write(commentsJson)